# Download Periodicals from The Talon Conspiracy
 
 This is a inteactive development notebook for building utilities to scrape periodicals from The Talon Conspiracy. 

Include Selenium as the core scraping utility.

In [12]:
# Add path
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [13]:
#  regular imports
from utils.selenium_resource import SeleniumResource
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import (
    StaleElementReferenceException,
    TimeoutException,
    NoSuchElementException,
)
from urllib.parse import urlparse
from typing import List, Optional
import logging
import time
from itertools import chain
# import pickle
from dataclasses import dataclass

Dataclasses to store metadata and important dowload link targets.

In [16]:
@dataclass
class IssuuItem:
    issuu_name: str
    issuu_url: str
    issuu_img_src: str
    issuu_download_path: Optional[str] = None


@dataclass
class TTCContent:
    ttc_content_title: str
    ttc_items: List[IssuuItem]


@dataclass
class TTCTag:
    "The Talon Conpiracy - Items by Tag"
    ttc_accessible_name: str
    ttc_tag_name: str
    ttc_tag_url: str
    # content is added after tag scrape
    ttc_tag_content: Optional[List[TTCContent]] = None

In [17]:

from datetime import datetime
from logging.handlers import RotatingFileHandler


def setup_logging(log_level=logging.INFO) -> logging.Logger:
    """
    Configure logging to both file and console with rotation and formatting.

    Args:
        log_level: The logging level to use (default: logging.INFO)
    """
    # Create logs directory if it doesn't exist
    log_file = os.path.join(os.getcwd(), 'ttc_scrape.log')

    # Create formatter
    formatter = logging.Formatter(
        '[%(asctime)s] %(levelname)s [%(name)s:%(lineno)s] %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )

    # Configure root logger
    root_logger = logging.getLogger()
    root_logger.setLevel(log_level)

    # Clear any existing handlers
    root_logger.handlers = []

    # Create rotating file handler (10MB max size, keep 5 backup files)
    file_handler = RotatingFileHandler(
        filename=log_file,
        maxBytes=10*1024*1024,  # 10MB
        backupCount=5,
        encoding='utf-8'
    )
    file_handler.setFormatter(formatter)
    file_handler.setLevel(log_level)

    # Create console handler
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)
    console_handler.setLevel(log_level)

    # Add handlers to root logger
    root_logger.addHandler(file_handler)
    root_logger.addHandler(console_handler)

    # Log system info at startup
    logger = logging.getLogger("TTC Scrape Logger")
    logger.info('='*50)
    logger.info(f'Logging initiated at {
                datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
    logger.info(f'Log file location: {log_file}')
    logger.info('='*50)
    return logger


class WebScraper:
    def __init__(self, max_retries: int = 3, timeout: int = 10):
        self.max_retries = max_retries
        self.timeout = timeout
        self.logger = setup_logging()

    def wait_and_find_element(
            self,
            driver,
            by: By,
            value: str,
            timeout: Optional[int] = None,
    ) -> Optional[any]:
        """Safely wait for and find an element with retries."""
        timeout = timeout or self.timeout
        for attempt in range(self.max_retries):
            try:
                return WebDriverWait(driver, timeout).until(
                    EC.presence_of_element_located((by, value))
                )
            except StaleElementReferenceException:
                if attempt == self.max_retries - 1:
                    raise
                time.sleep(1)
            except TimeoutException:
                self.logger.warning(
                    f"Timeout waiting for element {by}={value}")
                return None

    def wait_and_find_elements(
        self,
        driver,
        by: By,
        value: str,
        timeout: Optional[int] = None,
    ) -> List[any]:
        """Safely wait for and find elements with retries."""
        timeout = timeout or self.timeout
        for attempt in range(self.max_retries):
            try:
                elements = WebDriverWait(driver, timeout).until(
                    EC.presence_of_all_elements_located((by, value))
                )
                return elements
            except StaleElementReferenceException:
                if attempt == self.max_retries - 1:
                    raise
                time.sleep(1)
            except TimeoutException:
                self.logger.warning(
                    f"Timeout waiting for elements {by}={value}")
                return []

    def get_attribute_safely(self, element, attribute: str) -> Optional[str]:
        """Safely get an attribute from an element with retries."""
        for attempt in range(self.max_retries):
            try:
                return element.get_attribute(attribute)
            except StaleElementReferenceException:
                if attempt == self.max_retries - 1:
                    return None
                time.sleep(1)

    def click_download_button(
        self,
        driver,
        timeout: Optional[int] = None,
        wait_time: int = 5
    ) -> bool:
        """
        Click the download button with data-tooltip="Download" and wait for download to complete.

        Args:
            driver: Selenium WebDriver instance
            timeout: Optional timeout override (uses instance timeout if not specified)
            wait_time: Time to wait after clicking for download to complete (default: 5 seconds)

        Returns:
            bool: True if download button was found and clicked successfully, False otherwise
        """
        try:
            # Find download button using data-tooltip attribute
            download_button = self.wait_and_find_element(
                driver,
                By.CSS_SELECTOR,
                '[data-tooltip="Download"]',
                timeout
            )

            if not download_button:
                self.logger.warning("Download button not found")
                return False

            # Check if button is clickable
            if not download_button.is_enabled():
                self.logger.warning("Download button is not enabled")
                return False

            # Scroll button into view to ensure it's clickable
            driver.execute_script(
                "arguments[0].scrollIntoView(true);", download_button)
            time.sleep(1)  # Brief pause after scrolling

            # Click the download button
            download_button.click()
            self.logger.info("Download button clicked successfully")

            # Wait for specified time to allow download to complete
            time.sleep(wait_time)

            return True

        except Exception as e:
            self.logger.error(f"Error during download: {str(e)}")
            return False

In [18]:
def scrape_ttc_by_periodical_content(
    url: str,
    client=SeleniumResource(),
    scraper=WebScraper(),
) -> TTCContent:
    try:

        client.setup_for_execution()
        driver = client.driver
        # Navigate to tag page

        driver.get(url)

        # Get content results
        content_results = scraper.wait_and_find_elements(
            driver,
            By.CLASS_NAME,
            "results_content"
        )

        ttc_content = []
        for result in content_results:
            try:
                # Get the Category type. Include only periodicals.

                category_labels = [category.text for category in result.find_elements(
                    By.CSS_SELECTOR,
                    "h3 a[rel]"
                )]

                if "PERIODICALS" not in category_labels:
                    continue

                # Get title from the current result element
                result_title = result.find_element(By.TAG_NAME, "h1")
                if not result_title:
                    scraper.logger.warning(
                        "No title found for content result")
                    continue

                title_text = result_title.text
                scraper.logger.info(f"Found title for result {title_text}")

                # Get links with images
                links_with_images = result.find_elements(
                    By.CSS_SELECTOR,
                    "a:has(img)",
                )
                if not links_with_images:
                    scraper.logger.warning("No links with images")
                    continue

                ttc_issuus = []
                for link_img in links_with_images:
                    issuu_href = scraper.get_attribute_safely(
                        link_img, "href")
                    parsed_issuu_href = urlparse(issuu_href)
                    issuu_url = f"{parsed_issuu_href.scheme}://{parsed_issuu_href.netloc}{parsed_issuu_href.path}"

                    img_element = link_img.find_element(
                        By.TAG_NAME,
                        "img",
                    )
                    if not img_element:
                        continue

                    issuu_img_src = scraper.get_attribute_safely(
                        img_element, "src")
                    if not issuu_href or not issuu_img_src:
                        continue

                    issu_name = urlparse(issuu_img_src).path.strip(
                        "/").split("/")[-1]
                    issuu_item = IssuuItem(
                        issuu_name=issu_name,
                        issuu_url=issuu_url,
                        issuu_img_src=issuu_img_src
                    )
                    ttc_issuus.append(issuu_item)

                ttc_content_result = TTCContent(
                    ttc_content_title=result_title.text,
                    ttc_items=ttc_issuus.copy()
                )
                ttc_content.append(ttc_content_result)

            except StaleElementReferenceException:
                continue
            except NoSuchElementException:
                logging.warning("Missing h1 tag in result.")
                continue
            except StaleElementReferenceException:
                logging.warning(
                    "Result became stale, skipping")
                continue
        return ttc_content
    except Exception as e:
        scraper.logger.error(f"An error occurred: {str(e)}")
        return None
    finally:
        client.teardown_after_execution()


###################
periodical_urls = [f"https://thetalonconspiracy.com/category/periodicals/page/{
    page_num}/"for page_num in range(1, 11)]

# Unnest
periodicals = list(chain(*[scrape_ttc_by_periodical_content(
    url) for url in periodical_urls]))

[2024-12-07 19:09:52] INFO [TTC Scrape Logger:49] ==================================================
[2024-12-07 19:09:52] INFO [TTC Scrape Logger:50] Logging initiated at 2024-12-07 19:09:52
[2024-12-07 19:09:52] INFO [TTC Scrape Logger:52] Log file location: /app/nb/ttc_scrape.log
[2024-12-07 19:09:52] INFO [TTC Scrape Logger:53] ==================================================


[2024-12-07 19:10:17] INFO [TTC Scrape Logger:42] Found title for result FRONTLINE NEWS – THE COMPLETE SET!
[2024-12-07 19:10:18] INFO [TTC Scrape Logger:42] Found title for result HISTORY AS IT WAS MADE: A TROVE OF ANTI-HUNTINGDON LIFE SCIENCES NEWSLETTERS.
[2024-12-07 19:10:19] INFO [TTC Scrape Logger:42] Found title for result ANIMOSITY #3
[2024-12-07 19:10:19] INFO [TTC Scrape Logger:42] Found title for result BITE BACK (U.K.) #1&2
[2024-12-07 19:10:19] INFO [TTC Scrape Logger:42] Found title for result N.A.L.L. NEWSLETTERS – SUMMER AND SPRING 1985
[2024-12-07 19:10:19] INFO [TTC Scrape Logger:42] Found title for result TURNING POINT #7-8
[2024-12-07 19:10:19] INFO [TTC Scrape Logger:42] Found title for result EARTH FIRST! JOURNAL 1987
[2024-12-07 19:10:19] INFO [TTC Scrape Logger:42] Found title for result WILD ROCKIES REVIEW VOL. 2 #1-2
[2024-12-07 19:10:19] INFO [TTC Scrape Logger:42] Found title for result N.A.L.L. NEWSLETTER #7-8
[2024-12-07 19:10:30] INFO [TTC Scrape Logger:4

In [19]:
import pickle

# Saving an object to a pickle file
with open('periodicals.pkl', 'wb') as file:
    pickle.dump(periodicals, file)


In [22]:
def read_from_pickle(path):
    with open(path, 'rb') as file:
        try:
            while True:
                return pickle.load(file)
        except EOFError:
            pass

periodicals = read_from_pickle("./periodicals.pkl")



In [23]:
periodicals

[TTCContent(ttc_content_title='FRONTLINE NEWS – THE COMPLETE SET!', ttc_items=[IssuuItem(issuu_name='FrontLineNews1.jpg', issuu_url='https://issuu.com/conflictgypsy/docs/alffrontline_1', issuu_img_src='http://www.thetalonconspiracy.com/content/FrontLineNews1.jpg', issuu_download_path=None), IssuuItem(issuu_name='frontline2thumb.jpg', issuu_url='https://issuu.com/conflictgypsy/docs/frontline2', issuu_img_src='http://www.thetalonconspiracy.com/images/frontline2thumb.jpg', issuu_download_path=None), IssuuItem(issuu_name='frontline3.jpg', issuu_url='https://issuu.com/conflictgypsy/docs/frontline3', issuu_img_src='http://www.thetalonconspiracy.com/content/frontline3.jpg', issuu_download_path=None), IssuuItem(issuu_name='frontline4.jpg', issuu_url='https://issuu.com/conflictgypsy/docs/frontline-4', issuu_img_src='http://www.thetalonconspiracy.com/content/frontline4.jpg', issuu_download_path=None)]),
 TTCContent(ttc_content_title='HISTORY AS IT WAS MADE: A TROVE OF ANTI-HUNTINGDON LIFE SCIENC

In [24]:
periodicals
x = 1

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import random



class HumanLikeReader:
    def __init__(self):
        self.driver = None
        # Average human reading time per page in seconds (30-90 seconds)
        self.min_read_time = 30
        self.max_read_time = 90
        # Maximum number of pages to view in one session
        self.max_pages_per_session = 50

    def setup_driver(self):
        """Initialize the webdriver with human-like behavior"""
        options = Options()
        # Required arguments for running Chrome in a container
        options.add_argument('--headless')  # Run in headless mode
        # Required for running in container
        options.add_argument('--no-sandbox')
        # Overcome limited resource problems
        options.add_argument('--disable-dev-shm-usage')
        # Disable GPU hardware acceleration
        options.add_argument('--disable-gpu')

        # Add human-like characteristics
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_argument(
            '--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
        
        
            # Additional stability options
        options.add_argument('--window-size=1920,1080')
        options.add_argument('--disable-extensions')
        
        service = Service(
            executable_path="/usr/local/bin/chromedriver")
        service.start()

        self.driver = webdriver.Chrome(
            service=service,
            options=options
        )

    def simulate_human_viewing(self, url):
        """Access document with human-like reading patterns"""
        if not self.driver:
            self.setup_driver()

        try:
            # Navigate to the document
            self.driver.get(url)

            # Wait for the document to load
            WebDriverWait(self.driver, 20).until(
                EC.presence_of_element_located(
                    (By.CLASS_NAME, "document-content"))
            )

            # Simulate human-like reading behavior
            pages_viewed = 0
            while pages_viewed < self.max_pages_per_session:
                # Random reading time for each page
                reading_time = random.uniform(
                    self.min_read_time, self.max_read_time)
                time.sleep(reading_time)

                # Simulate occasional mouse movements
                self.simulate_mouse_movement()

                # Try to move to next page
                try:
                    next_button = self.driver.find_element(
                        By.CLASS_NAME, "next-page")
                    if not next_button.is_enabled():
                        break
                    next_button.click()
                    pages_viewed += 1
                except Exception as e:
                    break

                # Add random pauses between page turns
                time.sleep(random.uniform(2, 5))

        except Exception as e:
            print(f"Error during viewing: {str(e)}")
        finally:
            self.cleanup()

    def simulate_mouse_movement(self):
        """Simulate random mouse movements on the page"""
        actions = webdriver.ActionChains(self.driver)
        # Random movements within the viewport
        for _ in range(random.randint(2, 5)):
            x = random.randint(100, 800)
            y = random.randint(100, 600)
            actions.move_by_offset(x, y).perform()
            time.sleep(random.uniform(0.5, 2))

    def cleanup(self):
        """Clean up resources"""
        if self.driver:
            self.driver.quit()
            self.driver = None


def access_document(url):
    """Main function to access a document"""
    reader = HumanLikeReader(

    )
    reader.simulate_human_viewing(url)

# issuu_page = access_document(loaded_object[0].ttc_items[0].issuu_url)

NameError: name 'pickle' is not defined

In [7]:
def read_from_pickle(path):
    with open(path, 'rb') as file:
        try:
            while True:
                yield pickle.load(file)
        except EOFError:
            pass

periodicals = read_from_pickle("./periodicals.pkl")
# issuu_page = access_document(periodicals[0].ttc_items[0].issuu_url)

In [16]:
periodicals


KeyboardInterrupt: 